In [3]:
#START FULL CODE
import numpy as np
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.models import Model # basic class for specifying and training a neural network
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

batch_size = 32 # in each iteration, we consider 2 training examples at once
num_epochs = 1000 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 81 # the FC layer will have 81 neurons

training_data = np.array(
 [
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
 ]
)

y_train = np.array([[0], [1], [2], [3]])

# reshape from list to array
X_train = np.array([np.reshape(training_record, (9,9)) for training_record in training_data])
# add the depth dimension for keras
X_train = np.reshape(X_train, X_train.shape + (1,))
num_train, height, width, depth = X_train.shape
num_classes = np.unique(y_train).shape[0]

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
inp = Input(shape=(height, width, depth))

conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
 optimizer='adam', # using the Adam optimiser
 metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train, # Train the model using the training set...
 batch_size=batch_size, epochs=num_epochs,
 verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
print(model.metrics_names)
print(model.evaluate(X_train, Y_train, verbose=1)) # Evaluate the trained model on the training set

Train on 3 samples, validate on 1 samples
Epoch 1/1000
3/3 [==============================] - 0s - loss: 1.4046 - acc: 0.3333 - val_loss: 1.4451 - val_acc: 0.0000e+00
Epoch 2/1000
3/3 [==============================] - 0s - loss: 1.3400 - acc: 0.3333 - val_loss: 1.4975 - val_acc: 0.0000e+00
Epoch 3/1000
3/3 [==============================] - 0s - loss: 1.2821 - acc: 0.6667 - val_loss: 1.5265 - val_acc: 0.0000e+00
Epoch 4/1000
3/3 [==============================] - 0s - loss: 1.3214 - acc: 0.3333 - val_loss: 1.5820 - val_acc: 0.0000e+00
Epoch 5/1000
3/3 [==============================] - 0s - loss: 1.3271 - acc: 0.0000e+00 - val_loss: 1.6376 - val_acc: 0.0000e+00
Epoch 6/1000
3/3 [==============================] - 0s - loss: 1.1363 - acc: 0.3333 - val_loss: 1.7043 - val_acc: 0.0000e+00
Epoch 7/1000
3/3 [==============================] - 0s - loss: 1.3138 - acc: 0.3333 - val_loss: 1.7762 - val_acc: 0.0000e+00
Epoch 8/1000
3/3 [==============================] - 0s - loss: 1.1191 - acc: 0.

3/3 [==============================] - 0s - loss: 0.0100 - acc: 1.0000 - val_loss: 9.6231 - val_acc: 0.0000e+00
Epoch 66/1000
3/3 [==============================] - 0s - loss: 0.1010 - acc: 1.0000 - val_loss: 9.6974 - val_acc: 0.0000e+00
Epoch 67/1000
3/3 [==============================] - 0s - loss: 0.0208 - acc: 1.0000 - val_loss: 9.7719 - val_acc: 0.0000e+00
Epoch 68/1000
3/3 [==============================] - 0s - loss: 0.3999 - acc: 0.6667 - val_loss: 9.5949 - val_acc: 0.0000e+00
Epoch 69/1000
3/3 [==============================] - 0s - loss: 0.0359 - acc: 1.0000 - val_loss: 9.4090 - val_acc: 0.0000e+00
Epoch 70/1000
3/3 [==============================] - 0s - loss: 0.0588 - acc: 1.0000 - val_loss: 9.3021 - val_acc: 0.0000e+00
Epoch 71/1000
3/3 [==============================] - 0s - loss: 9.0512e-04 - acc: 1.0000 - val_loss: 9.2066 - val_acc: 0.0000e+00
Epoch 72/1000
3/3 [==============================] - 0s - loss: 0.0988 - acc: 1.0000 - val_loss: 9.1750 - val_acc: 0.0000e+00
Ep

3/3 [==============================] - 0s - loss: 0.0075 - acc: 1.0000 - val_loss: 14.1245 - val_acc: 0.0000e+00
Epoch 129/1000
3/3 [==============================] - 0s - loss: 0.0033 - acc: 1.0000 - val_loss: 14.1549 - val_acc: 0.0000e+00
Epoch 130/1000
3/3 [==============================] - 0s - loss: 0.0032 - acc: 1.0000 - val_loss: 14.1833 - val_acc: 0.0000e+00
Epoch 131/1000
3/3 [==============================] - 0s - loss: 0.1020 - acc: 1.0000 - val_loss: 14.2451 - val_acc: 0.0000e+00
Epoch 132/1000
3/3 [==============================] - 0s - loss: 7.6295e-06 - acc: 1.0000 - val_loss: 14.2983 - val_acc: 0.0000e+00
Epoch 133/1000
3/3 [==============================] - 0s - loss: 1.3891e-04 - acc: 1.0000 - val_loss: 14.3473 - val_acc: 0.0000e+00
Epoch 134/1000
3/3 [==============================] - 0s - loss: 4.1634e-04 - acc: 1.0000 - val_loss: 14.3926 - val_acc: 0.0000e+00
Epoch 135/1000
3/3 [==============================] - 0s - loss: 1.1921e-06 - acc: 1.0000 - val_loss: 14.43

3/3 [==============================] - 0s - loss: 1.0901e-04 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 191/1000
3/3 [==============================] - 0s - loss: 9.2221e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 192/1000
3/3 [==============================] - 0s - loss: 0.0039 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 193/1000
3/3 [==============================] - 0s - loss: 3.9736e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 194/1000
3/3 [==============================] - 0s - loss: 2.1431e-04 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 195/1000
3/3 [==============================] - 0s - loss: 1.3987e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 196/1000
3/3 [==============================] - 0s - loss: 7.7302e-04 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 197/1000
3/3 [==============================] - 0s - loss: 1.1971e-04 - acc: 1.0000 - val

3/3 [==============================] - 0s - loss: 3.6042e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 253/1000
3/3 [==============================] - 0s - loss: 1.0130e-04 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 254/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 255/1000
3/3 [==============================] - 0s - loss: 1.7484e-06 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 256/1000
3/3 [==============================] - 0s - loss: 1.3034e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 257/1000
3/3 [==============================] - 0s - loss: 2.1419e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 258/1000
3/3 [==============================] - 0s - loss: 7.3461e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 259/1000
3/3 [==============================] - 0s - loss: 2.9762e-04 - acc: 1.0000 -

...

3/3 [==============================] - 0s - loss: 0.0018 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 749/1000
3/3 [==============================] - 0s - loss: 1.2716e-06 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 750/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 751/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 752/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 753/1000
3/3 [==============================] - 0s - loss: 1.4623e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 754/1000
3/3 [==============================] - 0s - loss: 3.3776e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 755/1000
3/3 [==============================] - 0s - loss: 1.5895e-07 - acc: 1.0000 - val

3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 811/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 812/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 813/1000
3/3 [==============================] - 0s - loss: 3.9054e-04 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 814/1000
3/3 [==============================] - 0s - loss: 5.8954e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 815/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 816/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 817/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 -

3/3 [==============================] - 0s - loss: 2.7816e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 873/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 874/1000
3/3 [==============================] - 0s - loss: 2.5829e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 875/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 876/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 877/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 878/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 879/1000
3/3 [==============================] - 0s - loss: 2.5829e-07 - acc: 1.0000 -

3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 935/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 936/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 937/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 938/1000
3/3 [==============================] - 0s - loss: 4.3710e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 939/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 940/1000
3/3 [==============================] - 0s - loss: 9.7354e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 941/1000
3/3 [==============================] - 0s - loss: 1.3908e-07 - acc: 1.0000 -

3/3 [==============================] - 0s - loss: 3.6142e-05 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 997/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 998/1000
3/3 [==============================] - 0s - loss: 6.1393e-06 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 999/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 1000/1000
3/3 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
['loss', 'acc']
4/4 [==============================] - 0s
[4.0295238494873047, 0.75]


In [4]:
print(model.metrics_names)

['loss', 'acc']
